In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pydicom
from random import shuffle, seed
import copy
import cv2
import timm
import torch
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from torch.nn import BCELoss
from tqdm.notebook import tqdm
from torchinfo import summary

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
torch.manual_seed(42)
seed(42)

In [4]:
train_folder = 'train_images'
test_folder = 'test_images'

In [5]:
train_metadata = 'train.csv'
test_metadata = 'test.csv'

In [6]:
train_df = pd.read_csv(train_metadata)
test_df = pd.read_csv(test_metadata)

In [7]:
train_df.head()

site_id  patient_id    image_id laterality view   age  cancer  biopsy  \
0        2       10006   462822612          L   CC  61.0       0       0   
1        2       10006  1459541791          L  MLO  61.0       0       0   
2        2       10006  1864590858          R  MLO  61.0       0       0   
3        2       10006  1874946579          R   CC  61.0       0       0   
4        2       10011   220375232          L   CC  55.0       0       0   

   invasive  BIRADS  implant density  machine_id  difficult_negative_case  
0         0     NaN        0     NaN          29                    False  
1         0     NaN        0     NaN          29                    False  
2         0     NaN        0     NaN          29                    False  
3         0     NaN        0     NaN          29                    False  
4         0     0.0        0     NaN          21                     True

In [8]:
train_folders = os.listdir(train_folder)

In [9]:
train_folders[:5]

['33624', '64153', '65117', '19605', '40910']

In [10]:
train_df[train_df.patient_id==33624]

site_id  patient_id    image_id laterality view   age  cancer  biopsy  \
22215        1       33624  1111416410          L  MLO  56.0       0       0   
22216        1       33624  1462078398          L   CC  56.0       0       0   
22217        1       33624   788897862          R   CC  56.0       0       1   
22218        1       33624  1362210397          R  MLO  56.0       0       1   

       invasive  BIRADS  implant density  machine_id  difficult_negative_case  
22215         0     NaN        0       B          49                    False  
22216         0     NaN        0       B          49                    False  
22217         0     0.0        0       B          49                     True  
22218         0     0.0        0       B          49                     True

In [11]:
len(train_df['image_id'].value_counts()) == len(train_df)

True

Each image_id is unique so they can be used for assigning the scans to each fold

# Getting the CV-Setup done

The Training data will be split into 5 folds where each fold should contain roughtly the same number of cancer cases

For each patient the left and right breast can probably be placed in a different fold without causing any data leakage

The patient_id equals the folder in train_images and the image_id equals the scan inside the folder

In [12]:
# this function returns the image_ids for five folds with balanced cancer to non-cancer cases

def get_folds_with_image_ids(DF):
    
    df = DF.copy()
    
    patient_ids = list(train_df['patient_id'].unique())  # all unique patient_ids
    shuffle(patient_ids)
    
    # foldes to hold the image ids
    cancer_fold_1, cancer_fold_2, cancer_fold_3, cancer_fold_4, cancer_fold_5 = [], [], [], [], []
    no_cancer_fold_1, no_cancer_fold_2, no_cancer_fold_3, no_cancer_fold_4, no_cancer_fold_5 = [], [], [], [], []
    
    # list of folders
    cancer_folders = [cancer_fold_1, cancer_fold_2, cancer_fold_3, cancer_fold_4, cancer_fold_5]
    no_cancer_folders = [no_cancer_fold_1, no_cancer_fold_2, no_cancer_fold_3, no_cancer_fold_4, no_cancer_fold_5]
    
    # loop over all patient ids and assign them to a fold (each patient + side must be in a single fold)
    for ID in tqdm(patient_ids):
        for side in ['L', 'R']:
            filt = (df['patient_id'] == ID) & (df['laterality'] == side)  # boolean df as filter
            
            # df only with the current patient id & one breast
            current_df = df[filt]
            
            # image ids which should be assigned to one fold
            values_to_assign = current_df['image_id'].values
            
            # array of cancer values from the selected df
            cancer_value = current_df['cancer'].unique()
            
            # should only contain one value!
            if len(cancer_value) > 1:
                print('\n\n\nERROR: GOT DIFFERENT CANCER VALUES!\n\n\n')
            else:
                cancer_value = cancer_value[0]
            
            # the image ids should be assigned to the folder with the least values
            
            if cancer_value == 0:  # add it to one of no cancer folders...
                len_folders = [len(folder) for folder in no_cancer_folders]
                folder = np.array(len_folders).argmin()  # ...to the one with the least values in it
                for image_id in values_to_assign:
                    no_cancer_folders[folder].append(image_id)
                    
            # same with cancer_values
            elif cancer_value == 1:
                len_folders = [len(folder) for folder in cancer_folders]
                folder = np.array(len_folders).argmin()  # the one with the least values
                for image_id in values_to_assign:
                    cancer_folders[folder].append(image_id)
            
    # check if the folders are balanced
    len_cancer_folders = [len(folder) for folder in cancer_folders]
    len_no_cancer_folders = [len(folder) for folder in no_cancer_folders]
    print(f'Length of cancer folders: {len_cancer_folders}')
    print(f'Length of non cancer folders: {len_no_cancer_folders}\n')

    # check if the values got assigned correct
    counted_cancer_cases = sum([len(folder) for folder in cancer_folders])
    number_cancer_cases = len(df[df['cancer'] == 1])
    print(f'Number of cancer cases in total: {number_cancer_cases}')
    print(f'Number of cancer cases in cancer folders: {counted_cancer_cases}\n')

    counted_no_cancer_cases = sum([len(folder) for folder in no_cancer_folders])
    number_no_cancer_cases = len(df[df['cancer'] == 0])
    print(f'Number of non cancer cases in total: {number_no_cancer_cases}')
    print(f'Number of non cancer cases in non cancer folders: {counted_no_cancer_cases}\n')

    # add them up to five folds & shuffle them again
    fold_1, fold_2, fold_3, fold_4, fold_5 = [cancer + no_cancer for cancer, no_cancer in
                                              zip(cancer_folders, no_cancer_folders)]
    [shuffle(fold) for fold in [fold_1, fold_2, fold_3, fold_4, fold_5]]

    # print final length
    print(f'Final Length of Folds: {[len(fold) for fold in [fold_1, fold_2, fold_3, fold_4, fold_5]]}')
            
    return fold_1, fold_2, fold_3, fold_4, fold_5

In [13]:
%%time
fold_1, fold_2, fold_3, fold_4, fold_5 = get_folds_with_image_ids(train_df)

  0%|          | 0/11913 [00:00<?, ?it/s]

Length of cancer folders: [234, 232, 230, 232, 230]
Length of non cancer folders: [10711, 10709, 10709, 10710, 10709]

Number of cancer cases in total: 1158
Number of cancer cases in cancer folders: 1158

Number of non cancer cases in total: 53548
Number of non cancer cases in non cancer folders: 53548

Final Length of Folds: [10945, 10941, 10939, 10942, 10939]
CPU times: user 52.1 s, sys: 61.6 ms, total: 52.2 s
Wall time: 52.1 s


# Create the Datasets

The Datasets take as input the image_ids & the df and return the image and target

In [14]:
class Data(Dataset):
    
    # later gets initialized with one or four folds (cross-validation)
    def __init__(self, fold, DF):
        self.df = DF.copy()
        
        self.image_ids = copy.deepcopy(fold)  # dont shuffle original
        shuffle(self.image_ids)  # shuffle new image ids
        
        self.df_filtered = self.df[self.df['image_id'].isin(self.image_ids)]  # needs the same length for below
        
        # Problem: The new dataframe should be sorted with the new image_ids to get the correct other values
        self.df_sorted = self.df_filtered.iloc[np.argsort([self.image_ids.index(i) for i in 
                                                           self.df_filtered['image_id']])]
        
        # get the cancer value for each image id
        self.targets = list(self.df_sorted['cancer'].values)
        
        # also get the corresponding patient_ids for correct assignment below
        self.patient_ids = list(self.df_sorted['patient_id'].values)
                                              
        self.n_samples = len(self.image_ids)
        
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, index):
        
        # get the full path to the image_id of index...
        folder = train_folder
        patient_id = str(self.patient_ids[index])
        image_id = str(self.image_ids[index]) + '.dcm'  # each image_id is a .dcm file
        path_img = os.path.join(folder, patient_id, image_id)  # path to each image id
        
        img = pydicom.dcmread(path_img).pixel_array  # img as np.array
        
        # normalize it (values might be greater then 255 or less then 0)
        img = (img - img.min()) / (img.max() - img.min())
        
        img = cv2.resize(img, (512, 512))  # try 512 resolution first
        
        img_tensor = torch.tensor(img).type(torch.float32)
        
        img_tensor = torch.unsqueeze(img_tensor, 0)  # add rgb dim in shape index 0
        
        # to feed the grayscaled tensor into a pretrained rgb-model repeat the tensor 3 times in dim 0
        X = img_tensor.repeat(3, 1, 1)
        
        y = torch.tensor(self.targets[index])
        
        return X.to(device), y.to(device)

The data is highly unbalanced: 2 ways to fix this problem <br>
1) Assign cancer cases a higher loss <br>
2) Feed them more often (used approach)


In [20]:
def get_class_weights(DF):
    df = DF.copy()
    
    num_cancer_cases = len(df[df['cancer'] == 1])
    num_no_cancer_cases = len(df[df['cancer'] == 0])
    
    # the class weights just need to have a correct relative size
    # class weight for no cancer cases is set as 1 -> cancer cases weight is the number they appear less
    no_cancer_weight = 1
    cancer_weight = round(num_no_cancer_cases / num_cancer_cases, 2)
    
    print(f'Weights: No Cancer: {no_cancer_weight}; Cancer: {cancer_weight}')
    
    return [no_cancer_weight, cancer_weight]

In [21]:
class_weights = get_class_weights(train_df)

Weights: No Cancer: 1; Cancer: 46.24


The weighted random sampler needs to be created later, when the folds are made (it assigns each point a value)

# Create a first model using timm & transfer learning

In [34]:
model = timm.create_model('tf_efficientnetv2_xl_in21k', pretrained=True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_xl_in21k-fd7e8abf.pth" to /home/olli/.cache/torch/hub/checkpoints/tf_efficientnetv2_xl_in21k-fd7e8abf.pth


In [38]:
summary(model, input_data=torch.randn((1, 3, 512, 512)))

Layer (type:depth-idx)                        Output Shape              Param #
EfficientNet                                  [1, 21843]                --
├─Conv2dSame: 1-1                             [1, 32, 256, 256]         864
├─BatchNormAct2d: 1-2                         [1, 32, 256, 256]         64
│    └─Identity: 2-1                          [1, 32, 256, 256]         --
│    └─SiLU: 2-2                              [1, 32, 256, 256]         --
├─Sequential: 1-3                             [1, 640, 16, 16]          --
│    └─Sequential: 2-3                        [1, 32, 256, 256]         --
│    │    └─ConvBnAct: 3-1                    [1, 32, 256, 256]         9,280
│    │    └─ConvBnAct: 3-2                    [1, 32, 256, 256]         9,280
│    │    └─ConvBnAct: 3-3                    [1, 32, 256, 256]         9,280
│    │    └─ConvBnAct: 3-4                    [1, 32, 256, 256]         9,280
│    └─Sequential: 2-4                        [1, 64, 128, 128]         --
│    │ 

In [40]:
# make pretrained model weights untrainable
for param in model.parameters():
    param.requires_grad = False

In [54]:
# replace Linear part of the model with a fitting output
class Linear_Replacement(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.fc_1 = nn.Linear(1280, 128)
        self.bn_1 = nn.BatchNorm1d(128)
        self.act_1 = nn.ReLU()
        self.drop_1 = nn.Dropout(0.2)
        
        self.fc_2 = nn.Linear(128, 32)
        self.bn_2 = nn.BatchNorm1d(32)
        self.act_2 = nn.ReLU()
        self.drop_2 = nn.Dropout(0.2)
        
        self.fc_3 = nn.Linear(32, 1)
        self.out = nn.Sigmoid()
        
    def forward(self, x):

        x = self.fc_1(x)
        x = self.bn_1(x)
        x = self.act_1(x)
        x = self.drop_1(x)
        
        x = self.fc_2(x)
        x = self.bn_2(x)
        x = self.act_2(x)
        x = self.drop_2(x)
        
        x = self.fc_3(x)
        x = self.out(x)
        
        return x

In [55]:
model_lin_replacement = Linear_Replacement()

In [56]:
model.classifier = model_lin_replacement

In [57]:
summary(model, input_data=torch.randn((1, 3, 512, 512)))

Layer (type:depth-idx)                        Output Shape              Param #
EfficientNet                                  [1, 1]                    --
├─Conv2dSame: 1-1                             [1, 32, 256, 256]         (864)
├─BatchNormAct2d: 1-2                         [1, 32, 256, 256]         64
│    └─Identity: 2-1                          [1, 32, 256, 256]         --
│    └─SiLU: 2-2                              [1, 32, 256, 256]         --
├─Sequential: 1-3                             [1, 640, 16, 16]          --
│    └─Sequential: 2-3                        [1, 32, 256, 256]         --
│    │    └─ConvBnAct: 3-1                    [1, 32, 256, 256]         (9,280)
│    │    └─ConvBnAct: 3-2                    [1, 32, 256, 256]         (9,280)
│    │    └─ConvBnAct: 3-3                    [1, 32, 256, 256]         (9,280)
│    │    └─ConvBnAct: 3-4                    [1, 32, 256, 256]         (9,280)
│    └─Sequential: 2-4                        [1, 64, 128, 128]         

In [62]:
model.to(device)

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (2): ConvBnAct(
        

## Define optimizer, loss function & scheduler

In [64]:
# binary-crossentroy-loss
loss_bce = BCELoss()

In [67]:
optimizer = Adam(model.parameters(), lr=1e-4)

In [68]:
# reduce lr if there was no improvement after 10 epochs (should be 2 since its 5 fold-cv)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, verbose=True)

## Define functions for training and validation (loss + prob. f1-score)

In [71]:
'''Function for predicting a batch, calculating the loss,
   calculate the gradients & adjust weights'''

def train_batch(model, loss, optimizer, X, y):
    model.train()  # set model to training mode
    
    pred = model(X)  # prediction for current batch features
    
    pred = pred.squeeze()  # remove batch size dim
    
    batch_loss = loss(pred, y)  # calculate loss
    
    batch_loss.backward()  # backpropagate
    
    optimizer.step()  # adjust trainable weights
    
    optimizer.zero_grad()  # remove gradients for next batch
    
    return batch_loss.item()  # return loss value 

In [72]:

'''Function for calculating validation loss without adjusting the weights'''

@torch.no_grad()
def calculate_val_loss(model, loss, X, y):
    model.eval()  # evaluation mode
    
    pred = model(X)
    
    pred = pred.squeeze()
    
    batch_loss = loss(pred, y)
    
    return batch_loss.item()  # just return the loss for evaluation

The competition is evaluated with the probabilistic f1-score (pf1)

In [73]:
# provided implementation
def pfbeta(labels, predictions, beta):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0